In [1]:
import numpy as np
import pandas as pd

import sklearn.ensemble

from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')

train_data['date'] = train_data['date'].apply(lambda x: 12 * (int(x.split('-')[0]) - 2011) + int(x.split('-')[1]))
test_data['date'] = test_data['date'].apply(lambda x: 12 * (int(x.split('-')[0]) - 2011) + int(x.split('-')[1]))

In [3]:
train_data = train_data.replace(np.nan, -1)
test_data = test_data.replace(np.nan, -1)

clf_scaler = StandardScaler()
sgd_scaler = StandardScaler()

X_train_clf = clf_scaler.fit_transform(train_data.drop(columns=['id', 'price']).values)
X_train_sgd = sgd_scaler.fit_transform(train_data['date'].values.reshape(-1, 1))
y_train = train_data['price'].values

X_test_clf = clf_scaler.transform(test_data.drop(columns=['id']).values)
X_test_sgd = sgd_scaler.transform(test_data['date'].values.reshape(-1, 1))

In [4]:
clf = sklearn.ensemble.GradientBoostingRegressor(learning_rate=0.35, n_estimators=2200, min_samples_split=2)
clf.fit(X_train_clf, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.35, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=2200,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [5]:
sgd = SGDRegressor()
sgd.fit(X_train_sgd, y_train)

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [6]:
test_data['price_0'] = clf.predict(X_test_clf)
test_data['price'] = test_data['price_0'].values.reshape(-1, 1) * \
    (1 + (sgd.predict(X_test_sgd) / np.mean(y_train) - 1) / 2)

In [7]:
test_data[['id', 'price']].to_csv('submission.csv', index=False)